# Phylogenetics in Practice
## Preliminaries
Make sure to select the R kernel for this iPython notebook.

## Introduction

At the beginning of 2015, several cases of patients presenting symptoms of mild fever, rash, conjunctivitis and joint pain were reported by clinicians in South America. Investigation revealed infections with Zika virus, an obscure relative of dengue and chikungunya, that until the late 2000s had caused sporadic benign infections in Africa and Asia. Outbreaks emerged in Micronesia in 2007 and French Polynesia in 2013-14, during which an unusual increase in the incidence of severe neurological complications was first noticed. Over the course of 2015 and into 2016, clinicians in Brazil reported an upswing in the numbers of children born with microcephaly. The finger of blame was pointed towards Zika virus, a flavivirus, members of which group had been known to exhibit neurotropism and thereby carry the risk of causing neurological defects. As of March 2016, the epidemiological history and expectations for the ongoing spread of Zika virus were very uncertain.

The purpose of this practical is to demonstrate the tools available for phylogenetic analysis during early outbreak investigation. The aims are to address the following questions:
- What was the geographic origin of the 2015/2016 Zika outbreak?
- How was the virus related to previous Zika outbreaks?
- When did the 2015/2016 Zika virus strain diverge from other variants?
- Was there anything special about the 2015/2016 strain?
- At what speed was Zika virus spreading through South America?
- Could we have predicted the total number of cases?

## Genome Sequences

As of January 2016, only a modest number of Zika virus whole, or near-whole, genome sequences were available:

<iframe src="https://docs.google.com/spreadsheets/d/e/2PACX-1vS8yll6PC_F0ermshphnLg4ON21mMmGm3FXBOp9IRS3NlGG20L8VusTTryHzrf4PBAvNU1kN_TeK5Mo/pubhtml?widget=true&amp;headers=false" width="100%"></iframe>

Click [here](https://docs.google.com/spreadsheets/d/1tqKzU8rquKbkBbPqmRg6yW2jtt9EuQYjpHKq2mPAgaY/edit?usp=sharing) for the Google Doc. The first step in the analysis is to obtain the whole genome sequences from Genbank using the accession numbers provided in publications or found through a Genbank search query.

In your web browser, navigate to https://www.ncbi.nlm.nih.gov/genbank/
